# Notebook for preprocessing/reducing MCMC file sizes
- Steps to be followed after downloading the Goodwin, Lotka-Volterra, and Hinch data from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MDKNWM
- For Goodwin and Lotka experiments, the relevant files are titled theta.csv in the respective folders (theta denotes the 4 dimensional parameters), for each setting the csv file has 2M x 4 dimensional data (other files related to posterior, and gradient are not relevant for kernel thinning experiments)
    - There is a single posterior setting for each of Goodwin and Lotka model 
    - For each posterior sampling, 4 random walks, RW, Ada-RW, MALA, precond-MALA, are simulated for 2M iterations (over 4 dimensional parameter space)
    - We save each of the 8 csv files as pkl files
- For Hinch experiments, the data files have 'theta' in their file name (but on July 28 are part of Posterior.zip and Temperedposterior.zip in Cardiac folder)
    - There are two posteriors, posterior (Post) and tempered posterior (TP) (the temperature is 1 for Post, and 8 for TP)
    - For both Post and TP, a single chain, RW is simulated for 4M iterations (over 38 dimensional parameter space)
    - The csv files are 38 x 4M dimensional (making pre-processing a bit more involved since there are more columns than rows)
    - We first save transposed data as another csv file and then save the transposed data (4M by 38) as pkl for fast access; transposing makes it tractable for pd.read_csv to work with chunksize
    - We also save the Pnmax, and samples for this setting using burn-in of 1M samples as noted in Appendix S5.4 of the paper https://arxiv.org/abs/2005.03952 (v3)  (this is unlike Goodwin/LV where we simply saved the entire chain as pkl file)

In [ ]:
import pandas as pd
from tqdm import tqdm
import pickle as pkl
from sys import getsizeof
import numpy as np
import csv
import dask.dataframe
import os
import os.path
import time

In [ ]:
a = ! ls -LR | find . -name \theta\*.csv


# Goodwin and Lotka Files

- these files are small and can be dealth with easily directly in the memory
- of size 2M by 4 (iterations x dimensions)

In [ ]:
for name in a:
    if "Goodwin" in name or "Lotka" in name:
        pkl_name =  "../data/" + name.replace("/", "_")[6:-3] + "pkl" # find pkl file in data folder
        if os.path.exists(pkl_name):
            print(f'pkl file exists ({pkl_name})')
        else:
            print(f'loading {name} and dumping as pkl in {pkl_name}')
            data = np.zeros((int(2e6), 4))
            chunksize = 8000
            idx = int(0)
            for i, chunk in tqdm(enumerate(pd.read_csv(name, header=None, chunksize=chunksize))):
                n = chunk.shape[0]
                data[range(idx, idx+n), :] = chunk
                idx += n
            print(f'saving {name} as pkl file in {pkl_name}')
            with open(pkl_name, 'wb') as file:
                pkl.dump(data, file, protocol=pkl.HIGHEST_PROTOCOL)

# Cardiac/Hinch Data Files

- these files are huge, so need some better preprocessing
- By default saved as dimensions x iterations (38 x 4M)
- so we first save a transpose of it, and then convert it into a pkl file which are much faster to load

In [ ]:
filenames = ['./CSV/Hinch/Posterior/seed_1/theta_seed_1_temp_1.csv', './CSV/Hinch/Posterior/seed_2/theta_seed_2_temp_1.csv',
            './CSV/Hinch/Tempered_posterior/seed_1/theta_seed_1_temp_8.csv', './CSV/Hinch/Tempered_posterior/seed_2/theta_seed_2_temp_8.csv'
           ]
filenames_transposed = [f[:-4] + '_transposed.csv' for f in filenames]
pkl_names = [g[:-3] + 'pkl' for g in filenames_transposed]
for f, g, p in zip(filenames, filenames_transposed, pkl_names):
    print(f, g, p)

In [ ]:
# transpose to create efficient loaders with pandas
for f, g in zip(filenames, filenames_transposed):
    s = time.time()
    if os.path.exists(g):
        print(f'tranpose {g} exists for {f}')
        pass
    else:
        print(f'transposing {f} file with original shape 38 x 4M')
        a = zip(*csv.reader(open(f, "rt")))
        csv.writer(open(g, "wt")).writerows(a)
        print(f'This loop took {time.time()-s} seconds') # takes around 3 minutes on mac m1 first gen with 16GB ram

In [ ]:
for g, p in zip(filenames_transposed, pkl_names):
    if os.path.exists(p):
        print(f'Transposed pkl exists ({p})')
    else:
        print(f'loading {g}')
        data = np.zeros((int(4e6), 38))
        chunksize = 8000
        idx = int(0)
        for i, chunk in tqdm(enumerate(pd.read_csv(g, header=None, chunksize=chunksize))):
            n = chunk.shape[0]
            data[range(idx, idx+n), :] = chunk
            idx += n
        print(f'saving {g} as pkl file in {p}')
        with open(p, 'wb') as file:
            pkl.dump(data, file, protocol=pkl.HIGHEST_PROTOCOL)

## Further processing for Hinch data

- while loading the pkl file in memory is no longer an issue, it is huge in size so transferring it to cluster would take forever
- we preprocess Pnmax of size 2^15 and samples of sizes 4^m for m in {0, 1, ..., 7} by standard thinning from the end after burn-in of 1M samples

### Saving with normalizing (Hinch settings)

In [ ]:
pkl_names = ['./CSV/Hinch/Posterior/seed_1/theta_seed_1_temp_1_transposed.pkl', 
            './CSV/Hinch/Posterior/seed_2/theta_seed_2_temp_1_transposed.pkl',
            './CSV/Hinch/Tempered_posterior/seed_1/theta_seed_1_temp_8_transposed.pkl',
             './CSV/Hinch/Tempered_posterior/seed_2/theta_seed_2_temp_8_transposed.pkl'
            ]

prefixes = ["../data/" + f for f in ['Hinch_P_seed_1_temp_1', 'Hinch_P_seed_2_temp_1', 'Hinch_TP_seed_1_temp_8', 'Hinch_TP_seed_2_temp_8']]

for p, prefix in zip(pkl_names, prefixes):
    
    burn_in = int(1e6)
    print(f'\n loading {p}')
    with open(p, 'rb') as file:
        X = pkl.load(file)
    X = X[burn_in:]
    
    # separate in odd/even indices
    idx_even = np.arange(X.shape[0]-1, 1, -2)[::-1]
    idx_odd = np.arange(X.shape[0]-2, 0, -2)[::-1]
    assert(len(set(idx_even).intersection(set(idx_odd)))==0)

    # compute pnmax
    Xpnmax = X[idx_odd]
    end = Xpnmax.shape[0]
    nmax = int(2**15)
    step_size = int(end / nmax)
    assert(step_size>=1)
    # compute Pnmax by standard thinning from end
    idx_Pnmax = np.arange(end-1, 0, -step_size)[:nmax][::-1] # standard thin from the end
    filename = prefix + "_pnmax_15.pkl"
    print(f'saving pnmax of size {Xpnmax[idx_Pnmax].shape} to {filename}')
    with open(filename, "wb") as file:
        pkl.dump(Xpnmax[idx_Pnmax], file, protocol=pkl.HIGHEST_PROTOCOL)
        
    # compute samples
    X = X[idx_even]
    
    for m in range(8, 10):
        n = int(4**m)
        end = X.shape[0]
        # compute thinning parameter
        step_size = int(end / n)
        start = end-step_size*n
        assert(step_size>=1)
        assert(start>=0)
        samples = X[end-1:start:-step_size][::-1]
        filename = prefix + f"_samples_n_{n}.pkl"
        print(f'saving samples of size {samples.shape} to {filename}')
        with open(filename, "wb") as file:
            pkl.dump(samples, file, protocol=pkl.HIGHEST_PROTOCOL)
        

### Saving after normalizing (Hinch Scaled)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
pkl_names = ['./CSV/Hinch/Posterior/seed_1/theta_seed_1_temp_1_transposed.pkl', 
            './CSV/Hinch/Posterior/seed_2/theta_seed_2_temp_1_transposed.pkl',
            './CSV/Hinch/Tempered_posterior/seed_1/theta_seed_1_temp_8_transposed.pkl',
             './CSV/Hinch/Tempered_posterior/seed_2/theta_seed_2_temp_8_transposed.pkl'
            ]

prefixes_scaled = [f + '_scaled' for f in prefixes]

for p, prefix in zip(pkl_names, prefixes_scaled):
    
    burn_in = int(1e6)
    print(f'\n loading {p}')
    with open(p, 'rb') as file:
        X = pkl.load(file)
    
    # standardize the data
    
    X = X[burn_in:]
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X) # center and scale the data    
    # separate in odd/even indices
    idx_even =  np.arange(X.shape[0]-1, -1, -2)[::-1]
    idx_odd = np.arange(X.shape[0]-2, -1, -2)[::-1]
    assert(len(set(idx_even).intersection(set(idx_odd)))==0)

    # compute pnmax
    nmax = int(2**15)
    idx_Pnmax = np.linspace(0, len(idx_odd)-1,  nmax, dtype=int, endpoint=True)
    filename = prefix + "_pnmax_15.pkl"
    print(f'saving pnmax of len {len(idx_Pnmax)} to {filename}')
    with open(filename, "wb") as file:
        pkl.dump(X[idx_odd][idx_Pnmax], file, protocol=pkl.HIGHEST_PROTOCOL)
        
    # compute samples
    for m in range(0, 9):
        n = int(4**m)
        end = X.shape[0]
        idx_samples = np.linspace(0, len(idx_even)-1,  int(n), dtype=int, endpoint=True)
        filename = prefix + f"_samples_n_{n}.pkl"
        print(f'saving samples of len {len(idx_samples)} to {filename}')
        with open(filename, "wb") as file:
            pkl.dump(X[idx_even][idx_samples], file, protocol=pkl.HIGHEST_PROTOCOL)
        

## Computing median parameters 

In [ ]:
from scipy.spatial.distance import pdist
from util_sample import compute_mcmc_params_p, sample 

In [ ]:

med_dist_params = {} 
n = int(2**14)
for filename in ['Goodwin_RW', 'Goodwin_ADA-RW', 'Goodwin_MALA', 'Goodwin_PRECOND-MALA', 'Lotka_RW', 'Lotka_ADA-RW', 'Lotka_MALA', 'Lotka_PRECOND-MALA']:
    params_p = compute_mcmc_params_p(filename, nmax=int(2**15), include_last=True, profiling=False)
    X = sample(n, params_p)
    med_dist_params[filename] = np.nanmedian(pdist(X)).round(4)
    print(filename, X.shape, med_dist_params[filename])


In [ ]:
for filename in ['Hinch_P_seed_1_temp_1', 'Hinch_P_seed_2_temp_1', 
                                     'Hinch_TP_seed_1_temp_8', 'Hinch_TP_seed_2_temp_8']:
    f = "data/" + filename + '_scaled' + '_samples_n_16384.pkl'
    with open(f, "rb") as file:
        X = pkl.load(file)
    print(filename, X.shape, np.nanmedian(pdist(X)).round(4))
